<a href="https://colab.research.google.com/github/jaoaguiar097/IAalura_/blob/main/Pipelines_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install langchain langchain-google-genai "unstructured[pdf]" duckdb pandas
!pip install langchain_community
!pip install chromadb


In [16]:
import os
import duckdb
import pandas as pd
from datetime import datetime
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('chatbot')

In [17]:
from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("relatorio_vendas.pdf", mode="elements")

docs_unstructured = loader.load()

print(f"Total de elementos extraídos: {len(docs_unstructured)}\n")

for doc in docs_unstructured:
  print(f"---- TIPO DE ELEMENTO: {doc.metadata.get('category')} ---")
  print(doc.page_content)
  print("\n")

Total de elementos extraídos: 23

---- TIPO DE ELEMENTO: NarrativeText ---
Relatório Trimestral de Vendas - Q1 2024 Este relatório apresenta uma análise detalhada das vendas no primeiro trimestre de 2024. A performance geral foi positiva, com crescimento em todas as categorias de produtos. A seguir, uma tabela com os resultados por produto.


---- TIPO DE ELEMENTO: Title ---
ID Produto


---- TIPO DE ELEMENTO: Title ---
Nome do Produto


---- TIPO DE ELEMENTO: Title ---
Categoria Unidades Vendidas Receita (R$)


---- TIPO DE ELEMENTO: Title ---
PROD-001


---- TIPO DE ELEMENTO: Title ---
Laptop Pro X


---- TIPO DE ELEMENTO: Title ---
Eletrônicos


---- TIPO DE ELEMENTO: UncategorizedText ---
1500


---- TIPO DE ELEMENTO: UncategorizedText ---
7.500.000


---- TIPO DE ELEMENTO: Title ---
PROD-002 Cadeira Ergonômica Mobiliário


---- TIPO DE ELEMENTO: UncategorizedText ---
2500


---- TIPO DE ELEMENTO: UncategorizedText ---
1.250.000


---- TIPO DE ELEMENTO: Title ---
PROD-003


---- TI

In [18]:
from langchain_core.documents import Document

docs_com_metados = []

for doc in docs_unstructured:

  novos_metadados = doc.metadata.copy()

  novos_metadados['source'] = 'relatorio_vendas.pdf'
  novos_metadados['ingestion_date'] = datetime.now().strftime('%Y-%m-%d')
  novos_metadados['data_owner'] = 'Departamento de Vendas'

  docs_com_metados.append(
      Document(page_content=doc.page_content, metadata=novos_metadados)
  )

print(f"Total de documentos com metadados: {len(docs_com_metados)}\n")
print(docs_com_metados[-1])

Total de documentos com metadados: 23

page_content='A categoria de Eletrônicos continua a ser a mais lucrativa. A estratégia para o Q2 será focar em marketing para a Cadeira Ergonômica, que possui alto volume de vendas mas menor receita.' metadata={'source': 'relatorio_vendas.pdf', 'coordinates': {'points': ((78.0, 291.07), (78.0, 313.07), (502.6299999999997, 313.07), (502.6299999999997, 291.07)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'filename': 'relatorio_vendas.pdf', 'last_modified': '2026-01-06T19:35:31', 'page_number': 1, 'languages': ['por'], 'filetype': 'application/pdf', 'parent_id': '3fdda247a3f19f65b3ae66bc22fd1944', 'category': 'NarrativeText', 'element_id': '70b91eb7a59a15529f34b3b93f6d9ff0', 'ingestion_date': '2026-01-06', 'data_owner': 'Departamento de Vendas'}


In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50,
)

chunks = text_splitter.split_documents(docs_com_metados)

print(f"Numero de documentos original: {len(docs_com_metados)}")
print(f"Numero de chunks gerados: {len(chunks)}\n")

print(chunks[2])


Numero de documentos original: 23
Numero de chunks gerados: 23

page_content='Nome do Produto' metadata={'source': 'relatorio_vendas.pdf', 'coordinates': {'points': ((177.86, 171.07000000000005), (177.86, 181.07000000000005), (261.75, 181.07000000000005), (261.75, 171.07000000000005)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'filename': 'relatorio_vendas.pdf', 'last_modified': '2026-01-06T19:35:31', 'page_number': 1, 'languages': ['por'], 'filetype': 'application/pdf', 'category': 'Title', 'element_id': '8db19212a1dd1d7912af3dad01463352', 'ingestion_date': '2026-01-06', 'data_owner': 'Departamento de Vendas'}


In [20]:
import duckdb
import pandas as pd

# Conectar ao DuckDB (ele cria o arquivo se não existir)
con = duckdb.connect(database=':memory:', read_only=False)

# Criar uma tabela de produtos
con.execute("""
CREATE TABLE produtos (
    id INTEGER,
    nome VARCHAR,
    categoria VARCHAR,
    preco FLOAT,
    estoque INTEGER,
    descricao VARCHAR
);
""")

# Inserir dados de exemplo
produtos_df = pd.DataFrame({
    'id': [101, 102, 103, 104],
    'nome': ['Laptop Gamer Z', 'Mouse Óptico Fast', 'Teclado Mecânico Pro', 'Monitor Curvo 34"'],
    'categoria': ['Eletrônicos', 'Acessórios', 'Acessórios', 'Eletrônicos'],
    'preco': [9500.00, 250.00, 800.00, 3200.00],
    'estoque': [15, 120, 60, 25],
    'descricao': [
        'Laptop de alta performance com placa de vídeo dedicada e 32GB RAM.',
        'Mouse com 16.000 DPI e design ergonômico para longas sessões.',
        'Teclado com switches mecânicos, RGB e layout ABNT2.',
        'Monitor ultrawide com alta taxa de atualização e cores vibrantes.'
    ]
})

con.register('produtos_df', produtos_df)
con.execute('INSERT INTO produtos SELECT * FROM produtos_df')

print("Tabela 'produtos' criada e populada no DuckDB.")

# Verificar os dados
print(con.execute("SELECT * FROM produtos;").fetchdf())


Tabela 'produtos' criada e populada no DuckDB.
    id                  nome    categoria   preco  estoque  \
0  101        Laptop Gamer Z  Eletrônicos  9500.0       15   
1  102     Mouse Óptico Fast   Acessórios   250.0      120   
2  103  Teclado Mecânico Pro   Acessórios   800.0       60   
3  104     Monitor Curvo 34"  Eletrônicos  3200.0       25   

                                           descricao  
0  Laptop de alta performance com placa de vídeo ...  
1  Mouse com 16.000 DPI e design ergonômico para ...  
2  Teclado com switches mecânicos, RGB e layout A...  
3  Monitor ultrawide com alta taxa de atualização...  


In [21]:
# Query para selecionar os dados
df_produtos = con.execute("SELECT * FROM produtos;").fetchdf()

# Lista para armazenar os documentos
docs_sql = []

for _, row in df_produtos.iterrows():
    # Criar um texto descritivo a partir da linha
    page_content = f"Produto: {row['nome']}. Categoria: {row['categoria']}. Preço: R${row['preco']:.2f}. Em estoque: {row['estoque']} unidades. Descrição: {row['descricao']}"

    # Criar metadados estratégicos
    metadata = {
        'source': 'tabela_produtos_duckdb',
        'produto_id': row['id'],
        'categoria': row['categoria'],
        'preco': row['preco'],
        'ingestion_date': datetime.now().strftime('%Y-%m-%d')
    }

    docs_sql.append(
        Document(page_content=page_content, metadata=metadata)
    )

# Fechar a conexão com o banco
con.close()

print(f"Total de documentos gerados a partir do SQL: {len(docs_sql)}\n")
print("Exemplo de documento gerado a partir de uma linha do banco de dados:")
print(docs_sql[0])


Total de documentos gerados a partir do SQL: 4

Exemplo de documento gerado a partir de uma linha do banco de dados:
page_content='Produto: Laptop Gamer Z. Categoria: Eletrônicos. Preço: R$9500.00. Em estoque: 15 unidades. Descrição: Laptop de alta performance com placa de vídeo dedicada e 32GB RAM.' metadata={'source': 'tabela_produtos_duckdb', 'produto_id': 101, 'categoria': 'Eletrônicos', 'preco': 9500.0, 'ingestion_date': '2026-01-06'}


In [24]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata

documentos_finais = chunks + docs_sql

print(f"Total de documentos a serem indexados: {len(documentos_finais)}")

documentos_filtrados = filter_complex_metadata(documentos_finais)

print(f"Total de documentos filtrados: {len(documentos_filtrados)}")

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

vector_store = Chroma.from_documents(
    documents=documentos_filtrados,
    embedding=embeddings
)

Total de documentos a serem indexados: 27
Total de documentos filtrados: 27


In [28]:
doc.metadata

{'last_modified': '2026-01-06T19:35:31',
 'page_number': 1,
 'data_owner': 'Departamento de Vendas',
 'filetype': 'application/pdf',
 'filename': 'relatorio_vendas.pdf',
 'element_id': 'e45639bc59b5fc84d5f4e941cf87de20',
 'ingestion_date': '2026-01-06',
 'category': 'Title',
 'source': 'relatorio_vendas.pdf'}

In [30]:
#pergunta sobre o PDF

pergunta_pdf = "qual foi a receita com laptops?"

resultado_pdf = vector_store.similarity_search(pergunta_pdf , k=2)

print(f"pergunta: {pergunta_pdf}\n")

for doc in resultado_pdf:
  print(f"- Similaridade: {doc.page_content} ")
  print(f"-- Fonte: {doc.metadata.get('source')}, categoria: {doc.metadata.get('category')}")
  print("-"*20)

# pergunta sobre o banco de dados

pergunta_sql = "Me fale sobre o teclado mecanico"
resultado_sql = vector_store.similarity_search(pergunta_sql, k=2)

print(f"pergunta: {pergunta_sql}\n")

for doc in resultado_sql:
  print(f"- Similaridade: {doc.page_content} ")
  print(f"-- Fonte: {doc.metadata.get('source')}, categoria: {doc.metadata.get('category')}")
  print("-"*20)

#


pergunta: qual foi a receita com laptops?

- Similaridade: Produto: Laptop Gamer Z. Categoria: Eletrônicos. Preço: R$9500.00. Em estoque: 15 unidades. Descrição: Laptop de alta performance com placa de vídeo dedicada e 32GB RAM. 
-- Fonte: tabela_produtos_duckdb, categoria: None
--------------------
- Similaridade: A categoria de Eletrônicos continua a ser a mais lucrativa. A estratégia para o Q2 será focar em marketing para a Cadeira Ergonômica, que possui alto volume de vendas mas menor receita. 
-- Fonte: relatorio_vendas.pdf, categoria: NarrativeText
--------------------
pergunta: Me fale sobre o teclado mecanico

- Similaridade: Produto: Teclado Mecânico Pro. Categoria: Acessórios. Preço: R$800.00. Em estoque: 60 unidades. Descrição: Teclado com switches mecânicos, RGB e layout ABNT2. 
-- Fonte: tabela_produtos_duckdb, categoria: None
--------------------
- Similaridade: PROD-002 Cadeira Ergonômica Mobiliário 
-- Fonte: relatorio_vendas.pdf, categoria: Title
--------------------
